In [1]:
import pandas as pd

#### Dimension Discounts

In [ ]:
# Lit et affiche le contenu de l'ancien fichier discount.csv

data = pd.read_csv('Old_Data/discounts.csv', sep=",")
data.head()

,Start,End,Discont,Description,Category,Sub Category
0,2020-01-01,2020-01-10,0.4,40% discount during our New Year Winter Sale,Feminine,Coats and Blazers
1,2020-01-01,2020-01-10,0.4,40% discount during our New Year Winter Sale,Feminine,Sweaters and Knitwear
2,2020-01-01,2020-01-10,0.4,40% discount during our New Year Winter Sale,Masculine,Coats and Blazers
3,2020-01-01,2020-01-10,0.4,40% discount during our New Year Winter Sale,Masculine,Sweaters and Sweatshirts
4,2020-01-01,2020-01-10,0.4,40% discount during our New Year Winter Sale,Children,Coats


In [ ]:
# Récupération des champs utiles, suppression des doublons et génération de la clé primaire

df_Discounts = data[['Start', 'End', 'Discont', 'Category', 'Sub Category']]    
df_Discounts = df_Discounts.drop_duplicates()
df_Discounts = df_Discounts.reset_index(drop=True)
df_Discounts['Discount Key'] = df_Discounts.index +1

In [ ]:
# Affichage des données traitées

df_Discounts.head()

,Start,End,Discont,Category,Sub Category,Discount Key
0,2020-01-01,2020-01-10,0.4,Feminine,Coats and Blazers,1
1,2020-01-01,2020-01-10,0.4,Feminine,Sweaters and Knitwear,2
2,2020-01-01,2020-01-10,0.4,Masculine,Coats and Blazers,3
3,2020-01-01,2020-01-10,0.4,Masculine,Sweaters and Sweatshirts,4
4,2020-01-01,2020-01-10,0.4,Children,Coats,5


In [ ]:
# Écriture des données traités dans un nouveau fichier discounts.csv

df_Discounts.to_csv("Data/discounts.csv", index=False)

#### Dimension Stores

In [ ]:
# Lit et affiche le contenu de l'ancien fichier stores.csv

data = pd.read_csv('Old_Data/stores.csv', sep=",")
data.head()

,Store ID,Country,City,Store Name,Number of Employees,ZIP Code,Latitude,Longitude
0,1,United States,New York,Store New York,10,10001,40.7128,-74.0060
1,2,United States,Los Angeles,Store Los Angeles,8,90001,34.0522,-118.2437
2,3,United States,Chicago,Store Chicago,9,60601,41.8781,-87.6298
3,4,United States,Houston,Store Houston,10,77001,29.7604,-95.3698
4,5,United States,Phoenix,Store Phoenix,9,85001,33.4484,-112.0740


In [ ]:
# Récupération des champs utiles et suppression des doublons

df_Stores = data[['Store ID', 'Store Name', 'City', 'Country', 'ZIP Code', 'Latitude', 'Longitude']]
df_Stores = df_Stores.drop_duplicates()

In [ ]:
# Affichage des données traitées

df_Stores.head()

,Store ID,Store Name,City,Country,ZIP Code,Latitude,Longitude
0,1,Store New York,New York,United States,10001,40.7128,-74.0060
1,2,Store Los Angeles,Los Angeles,United States,90001,34.0522,-118.2437
2,3,Store Chicago,Chicago,United States,60601,41.8781,-87.6298
3,4,Store Houston,Houston,United States,77001,29.7604,-95.3698
4,5,Store Phoenix,Phoenix,United States,85001,33.4484,-112.0740


In [ ]:
# Écriture des données traités dans un nouveau fichier stores.csv

df_Stores.to_csv("Data/stores.csv", index=False)

#### Dimension Products

In [ ]:
# Lit et affiche le contenu de l'ancien fichier products.csv

data = pd.read_csv('Old_Data/products.csv', sep=",")
data.head()

,Product ID,Category,Sub Category,Description PT,Description DE,Description FR,Description ES,Description EN,Description ZH,Color,Sizes,Production Cost
0,1,Feminine,Coats and Blazers,Esportivo Veludo Verde Com Botões,Sport Samt Sport Mit Knöpfen,Sports Velvet Sports Avec Des Boutons,Deportes De Terciopelo Con Botones,Sports Velvet Sports With Buttons,运动天鹅绒运动与按钮,NaN,S|M|L|XL,10.73
1,2,Feminine,Sweaters and Knitwear,Luxuoso Denim Rosa Com Botões,Luxuriöser Rosa Jeans Mit Knöpfen,Léchard De Denim Rose Avec Boutons,Denim Rosa Lujoso Con Botones,Luxurious Pink Denim With Buttons,豪华的粉红色牛仔布和纽扣,PINK,S|M|L|XL,19.55
2,3,Feminine,Dresses and Jumpsuits,Retrô Tricot Preto Estampado,Black Tricot Gedruckter Tricot,Tricot Imprimé En Tricot Noir,Tricot Negro Tricot Impreso,Black Tricot Printed Tricot,黑色三角形印刷三角形,BLACK,S|M|L|XL,25.59
3,4,Feminine,Shirts and Blouses,Blusa De Algodão Básica,Basis -Baumwollbluse,Chemisier En Coton De Base,Blusa De Algodón,Basic Cotton Blouse,基本的棉衬衫,NaN,S|M|L|XL,27.62
4,5,Feminine,T-shirts and Tops,T-Shirt Básica De Algodão,Basis-Baumwoll-T-Shirt,T-Shirt En Coton De Base,Camiseta Básica De Algodón,Basic Cotton T-Shirt,基本棉T恤,NaN,S|M|L,11.69


In [ ]:
# Récupération des champs utiles et suppression des doublons

df_Products = data[['Product ID','Category', 'Sub Category', 'Description FR', 'Sizes', 'Production Cost']]
df_Products = df_Products.drop_duplicates()

In [ ]:
# Affichage des données traitées

df_Products.head()

,Product ID,Category,Sub Category,Description FR,Sizes,Production Cost
0,1,Feminine,Coats and Blazers,Sports Velvet Sports Avec Des Boutons,S|M|L|XL,10.73
1,2,Feminine,Sweaters and Knitwear,Léchard De Denim Rose Avec Boutons,S|M|L|XL,19.55
2,3,Feminine,Dresses and Jumpsuits,Tricot Imprimé En Tricot Noir,S|M|L|XL,25.59
3,4,Feminine,Shirts and Blouses,Chemisier En Coton De Base,S|M|L|XL,27.62
4,5,Feminine,T-shirts and Tops,T-Shirt En Coton De Base,S|M|L,11.69


In [ ]:
# Écriture des données traités dans un nouveau fichier products.csv

df_Products.to_csv("Data/products.csv", index=False)

#### Dimension Date

In [ ]:
# Lit et affiche le contenu de l'ancien fichier transactions.csv

data = pd.read_csv('Old_Data/transactions.csv', sep=",")
data.head()

,Invoice ID,Line,Customer ID,Product ID,Size,Color,Unit Price,Quantity,Date,Discount,Line Total,Store ID,Employee ID,Currency,Currency Symbol,SKU,Transaction Type,Payment Method,Invoice Total
0,INV-US-001-03558761,1,47162,485,M,NaN,80.5,1,2023-01-01 15:42:00,0.0,80.5,1,7,USD,$,MASU485-M-,Sale,Cash,126.7
1,INV-US-001-03558761,2,47162,2779,G,NaN,31.5,1,2023-01-01 15:42:00,0.4,18.9,1,7,USD,$,CHCO2779-G-,Sale,Cash,126.7
2,INV-US-001-03558761,3,47162,64,M,NEUTRAL,45.5,1,2023-01-01 15:42:00,0.4,27.3,1,7,USD,$,MACO64-M-NEUTRAL,Sale,Cash,126.7
3,INV-US-001-03558762,1,10142,131,M,BLUE,70.0,1,2023-01-01 20:04:00,0.4,42.0,1,6,USD,$,FECO131-M-BLUE,Sale,Cash,77.0
4,INV-US-001-03558762,2,10142,716,L,WHITE,26.0,1,2023-01-01 20:04:00,0.0,26.0,1,6,USD,$,MAT-716-L-WHITE,Sale,Cash,77.0


In [ ]:
# Récupération de la date, suppression des doublons et génération de la clé primaire

df_Date = data[['Date']] 
df_Date = df_Date.drop_duplicates()
df_Date = df_Date.reset_index(drop=True)
df_Date['Date Key'] = df_Date.index +1

In [ ]:
# Affichage des données traitées

df_Date.head()

,Date,Date Key
0,2023-01-01 15:42:00,1
1,2023-01-01 20:04:00,2
2,2023-01-01 20:47:00,3
3,2023-01-01 08:06:00,4
4,2023-01-01 17:25:00,5


In [ ]:
# Écriture des données traités dans un nouveau fichier date.csv

df_Date.to_csv("Data/date.csv", index=False)

#### Table de Fait

In [ ]:
# Récupération des champs utiles et suppression des doublons

df_Transactions = data[['Product ID', 'Store ID', 'Quantity', 'Unit Price', 'Line Total', 'Transaction Type', 'Invoice Total', 'Date']]    
df_Transactions = df_Transactions.drop_duplicates()

In [ ]:
# Affichage des données traitées

df_Transactions.head()

,Product ID,Store ID,Quantity,Unit Price,Line Total,Transaction Type,Invoice Total,Date
0,485,1,1,80.5,80.5,Sale,126.7,2023-01-01 15:42:00
1,2779,1,1,31.5,18.9,Sale,126.7,2023-01-01 15:42:00
2,64,1,1,45.5,27.3,Sale,126.7,2023-01-01 15:42:00
3,131,1,1,70.0,42.0,Sale,77.0,2023-01-01 20:04:00
4,716,1,1,26.0,26.0,Sale,77.0,2023-01-01 20:04:00


#### Réalisation des jointures

In [ ]:
# Jointure des Dataframes Transactions et Date dans la Table de Jointure

table_joint = pd.merge(df_Transactions, df_Date, on='Date', how='inner')
table_joint.head()

,Product ID,Store ID,Quantity,Unit Price,Line Total,Transaction Type,Invoice Total,Date,Date Key
0,485,1,1,80.5,80.5,Sale,126.7,2023-01-01 15:42:00,1
1,2779,1,1,31.5,18.9,Sale,126.7,2023-01-01 15:42:00,1
2,64,1,1,45.5,27.3,Sale,126.7,2023-01-01 15:42:00,1
3,131,1,1,70.0,42.0,Sale,77.0,2023-01-01 20:04:00,2
4,716,1,1,26.0,26.0,Sale,77.0,2023-01-01 20:04:00,2


In [ ]:
# Jointure du DataFrame Products avec la table de Jointure et supression des champs inutiles

table_joint2 = pd.merge(table_joint, df_Products, on='Product ID', how='inner')
table_joint2.drop(['Description FR','Sizes', 'Production Cost'], axis=1, inplace=True)
table_joint2.head()

,Product ID,Store ID,Quantity,Unit Price,Line Total,Transaction Type,Invoice Total,Date,Date Key,Category,Sub Category
0,485,1,1,80.5,80.5,Sale,126.7,2023-01-01 15:42:00,1,Masculine,Suits and Blazers
1,2779,1,1,31.5,18.9,Sale,126.7,2023-01-01 15:42:00,1,Children,Coats
2,64,1,1,45.5,27.3,Sale,126.7,2023-01-01 15:42:00,1,Masculine,Coats and Blazers
3,131,1,1,70.0,42.0,Sale,77.0,2023-01-01 20:04:00,2,Feminine,Coats and Blazers
4,716,1,1,26.0,26.0,Sale,77.0,2023-01-01 20:04:00,2,Masculine,T-shirts and Polos


In [ ]:
# Jointure du DataFrame Discounts avec la table de Jointure
# Éxécution longue (60s)

table_joint3 = pd.merge(table_joint2, df_Discounts, on=['Category','Sub Category'], how='inner')
table_joint3.head()

,Product ID,Store ID,Quantity,Unit Price,Line Total,Transaction Type,Invoice Total,Date,Date Key,Category,Sub Category,Start,End,Discont,Discount Key
0,2779,1,1,31.5,18.9,Sale,126.7,2023-01-01 15:42:00,1,Children,Coats,2020-01-01,2020-01-10,0.40,5
1,2779,1,1,31.5,18.9,Sale,126.7,2023-01-01 15:42:00,1,Children,Coats,2020-09-01,2020-09-15,0.45,26
2,2779,1,1,31.5,18.9,Sale,126.7,2023-01-01 15:42:00,1,Children,Coats,2021-01-01,2021-01-10,0.40,39
3,2779,1,1,31.5,18.9,Sale,126.7,2023-01-01 15:42:00,1,Children,Coats,2021-09-01,2021-09-15,0.45,60
4,2779,1,1,31.5,18.9,Sale,126.7,2023-01-01 15:42:00,1,Children,Coats,2022-01-01,2022-01-10,0.40,73


In [ ]:
# Affichage de la taille de la table de Jointure

table_joint3.shape

(46439839, 15)

In [ ]:
# Filtre du DataFrame de la table de Jointure pour ne garder que celle dont la date de la transaction appartient à la plage de la Promotion

df_Transactions = table_joint3[
    (table_joint3['Date'] >= table_joint3['Start']) &
    (table_joint3['Date'] <= table_joint3['End'])
]

In [ ]:
# Suppression de tout les champs qui ne sont plus utiles dans la table de Fait et Création de la clé primaire

df_Transactions.drop(['Date','Start', 'End', 'Discont', 'Category', 'Sub Category'], axis=1, inplace=True)
df_Transactions = df_Transactions.reset_index(drop=True)
df_Transactions['Transactions Key'] = df_Transactions.index +1
df_Transactions.head()

C:\Users\lucas\AppData\Local\Temp\ipykernel_15660\1890680766.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Transactions.drop(['Date','Start', 'End', 'Discont', 'Category', 'Sub Category'], axis=1, inplace=True)


,Product ID,Store ID,Quantity,Unit Price,Line Total,Transaction Type,Invoice Total,Date Key,Discount Key,Transactions Key
0,2779,1,1,31.5,18.9,Sale,126.7,1,107,1
1,64,1,1,45.5,27.3,Sale,126.7,1,105,2
2,131,1,1,70.0,42.0,Sale,77.0,2,103,3
3,1208,1,1,75.0,45.0,Sale,72.3,4,105,4
4,2639,1,1,45.5,27.3,Sale,72.3,4,106,5


In [ ]:
# Affichage de la taille de la table de jointure pour constater la suppression de toute les lignes et colonnes inutiles

df_Transactions.shape

(833700, 10)

In [ ]:
# Écriture des données traités dans un nouveau fichier transactions.csv

df_Transactions.to_csv("Data/transactions.csv", index=False)